In [ ]:
from tensorflow.keras.models import load_model
model_name = '226_labels0.9475'
loaded_model = load_model(f"../notebooks/models/Model_{model_name}.h5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '..notebooks/models/Model_226_labels0.9475.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [2]:
from keras.models import Model
from keras.layers import (Input, Bidirectional, LSTM, BatchNormalization, Dropout, 
                          MultiHeadAttention, Flatten, Dense, LeakyReLU, Conv1D, MaxPooling1D, GlobalAveragePooling1D)
from keras.regularizers import l2

input_shape = loaded_model.input_shape[1:]  # Get shape from loaded model
input_layer = Input(shape=input_shape)

#Convolution Layer
conv = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(input_layer)
conv = MaxPooling1D(pool_size=2)(conv)
# CPU-friendly LSTM (unroll=True, explicit activations)
lstm_out1 = Bidirectional(LSTM(256, return_sequences=True, activation='tanh',
                               recurrent_activation='sigmoid', unroll=True))(conv)
lstm_out1 = BatchNormalization()(lstm_out1)
lstm_out1 = Dropout(0.2)(lstm_out1)

lstm_out2 = Bidirectional(LSTM(256, return_sequences=True, activation='tanh',
                               recurrent_activation='sigmoid', unroll=True))(lstm_out1)
lstm_out2 = BatchNormalization()(lstm_out2)

lstm_out3 = Bidirectional(LSTM(256, return_sequences=True, activation='tanh',
                               recurrent_activation='sigmoid', unroll=True))(lstm_out2)
lstm_out3 = BatchNormalization()(lstm_out3)

attention_out = MultiHeadAttention(num_heads=4, key_dim=64)(lstm_out3, lstm_out3)

pooled=GlobalAveragePooling1D()(attention_out)

dense1 = Dense(128, kernel_regularizer=l2(1e-4))(pooled)
dense1 = LeakyReLU()(dense1)
dense1 = BatchNormalization()(dense1)
dense1 = Dropout(0.2)(dense1)

dense2 = Dense(64, kernel_regularizer=l2(1e-4))(dense1)
dense2 = LeakyReLU()(dense2)
dense2 = BatchNormalization()(dense2)
dense2 = Dropout(0.2)(dense2)

output_layer = Dense(loaded_model.output_shape[1], activation='softmax')(dense2)

model_cpu = Model(inputs=input_layer, outputs=output_layer)
model_cpu.set_weights(loaded_model.get_weights())

In [4]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model_cpu)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS
]
tflite_model = converter.convert()

with open(f"../models/{model_name}.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpxp3vwqs6/assets


INFO:tensorflow:Assets written to: /tmp/tmpxp3vwqs6/assets


Saved artifact at '/tmp/tmpxp3vwqs6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 543, 3), dtype=tf.float32, name='keras_tensor_41')
Output Type:
  TensorSpec(shape=(None, 255), dtype=tf.float32, name=None)
Captures:
  127604964367824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604964365360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604964375392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604965052960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604965059120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604964736016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604964721232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604964736192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604964725808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  127604964722992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12760496485723

W0000 00:00:1744607845.439187  148060 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1744607845.439204  148060 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-14 10:47:25.439361: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpxp3vwqs6
2025-04-14 10:47:25.461408: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-14 10:47:25.461422: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpxp3vwqs6
2025-04-14 10:47:25.859116: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-14 10:47:26.593339: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpxp3vwqs6
2025-04-14 10:47:27.243394: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 1804037 microseconds.
